In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

from loader import train_loader, val_loader, test_loader, scaler

In [2]:
class MixtureDensityForecastNet(nn.Module):
    def __init__(self, window_size, num_series, static_dim, latent_dim=128, 
                 n_mixtures=3, dropout=0.1, output_dim=2):
        """
        - window_size: số bước lịch sử.
        - num_series: số biến trong chuỗi (ví dụ: 2 cho Units và Revenue).
        - static_dim: số đặc trưng ngoại lai.
        - latent_dim: kích thước tầng ẩn của mạng FC.
        - n_mixtures: số thành phần hỗn hợp (mixture components).
        - output_dim: số biến dự báo (2).
        
        Đầu vào của mô hình là: flatten(x_seq) concat static_features.
        Kích thước đầu vào = window_size * num_series + static_dim.
        Output của mạng là các tham số của hỗn hợp Gaussian:
          - pi: mixing coefficients (n_mixtures)
          - mu: trung bình (n_mixtures x output_dim)
          - sigma: độ lệch chuẩn (n_mixtures x output_dim)
        Tổng output dimension = n_mixtures*(1 + 2*output_dim).
        """
        super(MixtureDensityForecastNet, self).__init__()
        self.window_size = window_size
        self.num_series = num_series
        self.static_dim = static_dim
        self.input_dim = window_size * num_series + static_dim
        self.n_mixtures = n_mixtures
        self.output_dim = output_dim
        
        self.fc = nn.Sequential(
            nn.Linear(self.input_dim, latent_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(latent_dim, latent_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        # Final layer: outputs n_mixtures*(1+2*output_dim)
        self.fc_out = nn.Linear(latent_dim, n_mixtures * (1 + 2 * output_dim))
    
    def forward(self, x_seq, x_cal):
        # x_seq: (batch, window_size, num_series)
        # x_cal: (batch, static_dim)
        batch_size = x_seq.size(0)
        x_seq_flat = x_seq.view(batch_size, -1)  # (batch, window_size*num_series)
        x_input = torch.cat([x_seq_flat, x_cal], dim=1)  # (batch, input_dim)
        h = self.fc(x_input)
        out = self.fc_out(h)
        # Reshape out: (batch, n_mixtures, 1+2*output_dim)
        out = out.view(batch_size, self.n_mixtures, 1 + 2 * self.output_dim)
        # Tách các tham số:
        pi = out[:, :, 0]  # (batch, n_mixtures)
        mu = out[:, :, 1:1+self.output_dim]  # (batch, n_mixtures, output_dim)
        sigma = out[:, :, 1+self.output_dim:]  # (batch, n_mixtures, output_dim)
        # Xử lý các tham số:
        pi = torch.softmax(pi, dim=1)
        sigma = torch.exp(sigma)  # đảm bảo sigma > 0
        return pi, mu, sigma

def mdn_loss(pi, mu, sigma, y_true):
    """
    Tính negative log-likelihood cho hỗn hợp Gaussian.
    - pi: (batch, n_mixtures)
    - mu: (batch, n_mixtures, output_dim)
    - sigma: (batch, n_mixtures, output_dim)
    - y_true: (batch, output_dim)
    """
    batch_size, n_mixtures, output_dim = mu.shape
    # Mở rộng y_true thành shape (batch, n_mixtures, output_dim)
    y_true = y_true.unsqueeze(1).expand_as(mu)
    
    # Tính probability density của từng thành phần Gaussian (giả sử độc lập các biến)
    # density = 1/(sqrt(2pi)*sigma) * exp(-0.5*((y - mu)/sigma)^2)
    exp_term = -0.5 * (((y_true - mu) / sigma) ** 2)
    coef = 1.0 / (np.sqrt(2 * np.pi) * sigma)
    component_prob = coef * torch.exp(exp_term)  # (batch, n_mixtures, output_dim)
    
    # Tính product theo output_dim (với giả sử độc lập)
    component_prob = torch.prod(component_prob, dim=2)  # (batch, n_mixtures)
    
    # Tính tổng có trọng số theo pi
    weighted_prob = pi * component_prob  # (batch, n_mixtures)
    prob = torch.sum(weighted_prob, dim=1)  # (batch)
    
    # Negative log likelihood
    nll = -torch.log(prob + 1e-8)  # tránh log(0)
    return torch.mean(nll)


In [3]:
model = MixtureDensityForecastNet(window_size=30, num_series=2, static_dim=18,
                                  latent_dim=64, n_mixtures=3,
                                  dropout=0.1, output_dim=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [4]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
kl_start = 0.0
kl_max = 0.001

In [ ]:
epochs = 1
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for x_seq, x_cal, y in train_loader:
        x_seq = x_seq.to(device)
        x_cal = x_cal.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        pi, mu, sigma = model(x_seq, x_cal)
        loss = mdn_loss(pi, mu, sigma, y)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item() * x_seq.size(0)
    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)
    
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for x_seq, x_cal, y in val_loader:
            x_seq = x_seq.to(device)
            x_cal = x_cal.to(device)
            y = y.to(device)
            pi, mu, sigma = model(x_seq, x_cal)
            loss = mdn_loss(pi, mu, sigma, y)
            running_val_loss += loss.item() * x_seq.size(0)
    epoch_val_loss = running_val_loss / len(val_loader.dataset)
    val_losses.append(epoch_val_loss)
    
    scheduler.step(epoch_val_loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")
    
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        checkpoint = {
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': epoch_val_loss
        }
        torch.save(checkpoint, 'best_mdn_checkpoint.pth')
        print(f"Best model updated at epoch {epoch+1} with validation loss {epoch_val_loss:.4f}")

Best model updated at epoch 1 with validation loss 1.2891
Best model updated at epoch 2 with validation loss 0.9108
Best model updated at epoch 3 with validation loss 0.7713
Best model updated at epoch 4 with validation loss 0.6073
Best model updated at epoch 5 with validation loss 0.4329
Best model updated at epoch 6 with validation loss 0.3327
Best model updated at epoch 7 with validation loss 0.2484
Best model updated at epoch 8 with validation loss 0.1099
Epoch [10/100], Train Loss: 0.1084, Val Loss: -0.0528
Best model updated at epoch 10 with validation loss -0.0528
Best model updated at epoch 13 with validation loss -0.1984
Best model updated at epoch 14 with validation loss -0.2087
Best model updated at epoch 15 with validation loss -0.2525
Best model updated at epoch 16 with validation loss -0.2707
Best model updated at epoch 17 with validation loss -0.2856
Best model updated at epoch 18 with validation loss -0.4104
Epoch [20/100], Train Loss: -0.3871, Val Loss: -0.3947
Best mo

In [6]:
model.eval()
test_preds = []
test_actuals = []
with torch.no_grad():
    for x_seq, x_cal, y in test_loader:
        x_seq = x_seq.to(device)
        x_cal = x_cal.to(device)
        y = y.to(device)
        pi, mu, sigma = model(x_seq, x_cal)
        # Dự báo điểm: ta có thể sử dụng weighted sum của μ theo pi
        pred = torch.sum(pi.unsqueeze(2) * mu, dim=1)  # (batch, output_dim)
        test_preds.append(pred.cpu().numpy())
        test_actuals.append(y.cpu().numpy())
test_preds = np.concatenate(test_preds, axis=0)
test_actuals = np.concatenate(test_actuals, axis=0)

# Inverse transform nếu cần (giả sử scaler đã được fit trên target)
test_preds_inv = scaler.inverse_transform(test_preds)
test_actuals_inv = scaler.inverse_transform(test_actuals)

r2 = r2_score(test_actuals_inv, test_preds_inv)
mape = mean_absolute_percentage_error(test_actuals_inv, test_preds_inv)
rmse = np.sqrt(mean_squared_error(test_actuals_inv, test_preds_inv))
print(f"Test R-squared: {r2:.4f}")
print(f"Test MAPE: {mape:.4f}")
print(f"Test RMSE: {rmse:.4f}")


Test R-squared: 0.9491
Test MAPE: 0.3406
Test RMSE: 168429.0625
